In [116]:
#@title STEP 1: Start here by selecting an Excel MST file exported by the Nanotemper software.
#@markdown To activate a step press the Play button on the left. Run the steps sequentially, but chose only one alternative if alternative steps are offered.<p>
#@markdown Please cite this article: <p>
#@markdown Atsarina Larasati Anindya, Maria-Jose Garcia-Bonete, Maja Jensen, Christian V. Recktenwald, Maria Bokarewa and Gergely Katona <i>Bayesian Progress Curve Analysis of MicroScale Thermophoresis Data.</i>
#@markdown (2022) <b><i>Digital discovery</i></b><p>
#@markdown https://doi.org/10.1039/D1DD00026H
# (c) 2022 Gergely Katona <gergely.katona@gu.se>
import io
import pandas as pd
#from google.colab import files
import numpy as np
import pymc3 as pm
import pylab as plt
import theano.tensor as T
from pymc3.math import switch

#Experimental data description 
#uploaded = files.upload()

#df=pd.read_excel(io.BytesIO(uploaded[list(uploaded.keys())[0]]))
df=pd.read_excel('wt_surv_dimer_MSTTraceRawData_Low.xlsx')


ligpos=df.loc[df.iloc[:,0] == 'Ligand Concentration:'].index[0]
firsttimepos=df.loc[df.iloc[:,0] == 'Time [s]'].index[0]
targind=df.loc[df.iloc[:,0] == 'TargetConcentration:'].index[0]
FluoConc=df.iloc[targind,1]

cideal=df.iloc[ligpos,1::3]
onedtime=df.iloc[firsttimepos+1:,0].reset_index(drop=True)
firstIRtimepos=onedtime.loc[onedtime==0].index[0]
fluorescence=df.iloc[firsttimepos+1:,1::3].reset_index(drop=True)
fluorescence=fluorescence.T.reset_index(drop=True).T
Capillary=len(cideal)
start_t=onedtime.iloc[0]
end_t=onedtime.iloc[-1]
print ("The data starts at position %i, which corresponds to %f s." % (0,start_t))
print ("The data ends at position %i, which corresponds to %f s." % (len(onedtime),end_t))
print ("The IR laser is supposed to be ON from position %i, which corresponds to 0 s."% (firstIRtimepos))
print ("The target concentration is %E ."% FluoConc)
if FluoConc < 1:
  print ("I guess you are using units of M?")
if FluoConc > 1:
  print ("I guess you are using units of nM?")
print ("%i capillaries were used." % Capillary)
print ("The ligand concentrations are %s . "% list(cideal))
print ("Units hopefully are in nM (due to the numerical limits of KD prior) and the KD will be expressed in the same unit. If these concentrations are invalid, please edit them in the input excel file.")

The data starts at position 0, which corresponds to -5.514154 s.
The data ends at position 543, which corresponds to 34.947815 s.
The IR laser is supposed to be ON from position 73, which corresponds to 0 s.
The target concentration is 9.700000E+01 .
I guess you are using units of nM?
16 capillaries were used.
The ligand concentrations are [761000, 380500, 190250, 95125, 47562.5, 23781.25, 11890.625, 5945.3125, 2972.6563, 1486.3281, 743.1641, 371.582, 185.791, 92.8955, 46.4478, 23.2239] . 
Units hopefully are in nM (due to the numerical limits of KD prior) and the KD will be expressed in the same unit. If these concentrations are invalid, please edit them in the input excel file.


In [117]:
#@title STEP 2: Displaying and verifying the data. Adjusting data range and laser start. { run: "auto" }
import ipywidgets as widgets
from IPython.display import display
#@markdown Step 2.1: Inspect the progress curve. Is this the data you want to model? The program tries to guess a suitable end point automatically, by choosing the minimum of Fnorm. This initial guess will vary from capillary to capillary, but once you start modelling the last displayed end data point will be used for for all capillaries. Please feel free to override data range according to your wishes, keep in mind that linear kinetic processes can increase Fnorm after the exponential processes finished and the model has a chance to deal with them. In that case, feel free to extend the data range beyond the time Fnorm reaches the absolute minimum.<p>
#@markdown Step 2.2: Zoom in with the sliders to the laser start point and adjust to the point where the Fnorm starts to drop rapidly. Experience shows that there is a few ms delay between time 0 of the instrument and the first sign of response in Fnorm. An empirical delay is already automatically applied to the laser start.<p>
#@markdown Step 2.3: Reset the end slider to the end of the IR irradiation period or any point earlier. The model does not take into account the recovery phase after the IR laser is swiched off.
Cap_display = "1" #@param ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16"]

Data_range=widgets.IntRangeSlider(
    value=[0, fluorescence[int(Capillary)-1].astype(float).idxmin()],
    min=0,
    max=len(onedtime),
    step=1,
    description='Data_range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)


IR_pos=widgets.IntSlider(
    value=firstIRtimepos+2,
    min=0,
    max=len(onedtime),
    step=1,
    description='IR_pos:',
    disabled=False
)

def f(Data_range, IR_pos):
    Start_pos,End_pos=Data_range
    fig, ax = plt.subplots()
    ax.plot(onedtime[Start_pos:End_pos],fluorescence[int(Cap_display)-1].astype(float).iloc[Start_pos:End_pos], color='gray')
    ax.annotate('IR laser', 
            xy=(onedtime[IR_pos], 1), 
            xytext=(onedtime[IR_pos], 0.95), 
            arrowprops = dict(facecolor='red', shrink=0.05)
    )
    plt.xlabel ("Time (s)")
    plt.ylabel (r"$F_{norm}$")
out = widgets.interactive_output(f, {'Data_range': Data_range, 'IR_pos': IR_pos})

widgets.HBox([widgets.VBox([Data_range, IR_pos]), out])



In [118]:
#@title STEP 3.1 (Alternative 1): Perform the MCMC sampling (linear phase starts before the IR irradiation)
#@markdown Use scenario: The fluroescence signal (Fnorm) is not stable at 1 before IR irradiation. Possible reasons: photodamage, unstable protein, not fully equilibrated target:ligand interaction.

#@markdown Choose the concentration units to provide appropriate prior distribution for K_D.

K_D_prior="informative uniform prior for nM units" #@param ["informative uniform prior for nM units", "informative uniform prior for M units", "flat exponential prior for any unit"]

#@markdown Enter the number of tuning and sampling steps for the Markov Chain Monte Carlo algorithm. Decrease these only if you know what you are doing.
Tuning = 1000 #@param {type:"number"}
Samples = 2000 #@param {type:"number"}

IR=IR_pos.value
start,end=Data_range.value
cap=int(Capillary)
time=np.array([onedtime[:,]]*cap,dtype=float).transpose()
with pm.Model() as model:
    if K_D_prior=="informative uniform prior for nM units":
        K_D = pm.Uniform('K_D', 1, 1e6)
    if K_D_prior=="informative uniform prior for M units":
        K_D = pm.Uniform('K_D', 1e-9, 1e-3)
    if K_D_prior=="flat exponential prior for any unit":
        K_D = pm.Exponential('K_D', 1e-3)
    BoundedNormal = pm.Bound(pm.Normal, lower=0, upper=1e8)
    c_fl = BoundedNormal('c_fl',mu=FluoConc,sd=FluoConc/10.0)

    U=pm.Beta('U',alpha=1,beta=1)
    B=pm.Beta('B',alpha=1,beta=1)
    A_total=pm.Deterministic('A_total', U + (B-U)*((c_fl + cideal.astype(float) + K_D - pm.math.sqrt(pm.math.sqr(c_fl + cideal.astype(float) + K_D) - 4.0*c_fl*cideal.astype(float)))/(2*c_fl)))
    
    L=pm.Uniform('L',-1,1, shape=cap)
    I=pm.Deterministic('I',1+L*start_t-A_total)
    R=pm.Beta('R',alpha=2,beta=1, shape=cap)
    A_1=pm.Deterministic('A_1',A_total*R)
    A_2=pm.Deterministic('A_2',A_total*(1-R))
    rate1=pm.Lognormal('rate1',0,1, shape=cap)
    rate2=pm.Lognormal('rate2',0,1, shape=cap)

    linearphase=L*(time-start_t)
    E_1=A_1*pm.math.exp(-1.0*rate1*time)
    E_2=A_2*pm.math.exp(-1.0*rate2*time)

    epsilon = pm.Lognormal('epsilon', 0, 1)
    laseron=np.tile(np.arange(0, len(onedtime)), (cap,1)).T

    pr = switch(IR >= laseron, 1.0+linearphase, linearphase+I+E_1+E_2)


    P = pm.Normal('P', mu=pr[start:end], sd=epsilon, observed=fluorescence.astype(float).iloc[start:end])


with model:
    trace = pm.sampling.sample(Samples, tune=Tuning,
                      init='ADVI'
                     )
    
with model:
    ppc = pm.sample_posterior_predictive(trace, samples=1000 ,model=model)

Auto-assigning NUTS sampler...
Initializing NUTS using advi...


Convergence achieved at 70400
Interrupted at 70,399 [35%]: Average Loss = 5.9551e+11
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [epsilon, rate2, rate1, R, L, B, U, c_fl, K_D]


Sampling 4 chains for 1_000 tune and 921 draw iterations (4_000 + 3_684 draws total) took 316 seconds.
The number of effective samples is smaller than 25% for some parameters.
/home/gergely/anaconda3/lib/python3.7/site-packages/pymc3/sampling.py:1618: UserWarning: samples parameter is smaller than nchains times ndraws, some draws and/or chains may not be represented in the returned posterior predictive sample
  "samples parameter is smaller than nchains times ndraws, some draws "


In [128]:
#@title STEP 3.2 (Alternative 2): Perform the MCMC sampling (linear phase starts at time 0)
#@markdown Use scenario: Fnorm is stable at 1 before IR irradiation, but after irradiation there is a trendlike increase or decrease in at least some of the progress curves. Possible causes: temperature induced denaturation of proteins, direct IR absorption in protein results in an electrodynamic interaction between the target and the ligand.

#@markdown Choose the concentration units to provide appropriate prior distribution for K_D.

K_D_prior="informative uniform prior for nM units" #@param ["informative uniform prior for nM units", "informative uniform prior for M units", "flat exponential prior for any unit"]


#@markdown Enter the number of tuning and sampling steps for the Markov Chain Monte Carlo algorithm. Decrease these only if you know what you are doing.
Tuning = 1000 #@param {type:"number"}
Samples = 2000 #@param {type:"number"}

IR=IR_pos.value
start,end=Data_range.value
cap=int(Capillary)
time=np.array([onedtime[:,]]*cap,dtype=float).transpose()
with pm.Model() as model:
    if K_D_prior=="informative uniform prior for nM units":
        K_D = pm.Uniform('K_D', 1, 1e6)
    if K_D_prior=="informative uniform prior for M units":
        K_D = pm.Uniform('K_D', 1e-9, 1e-3)
    if K_D_prior=="flat exponential prior for any unit":
        K_D = pm.Exponential('K_D', 1e-3)
    BoundedNormal = pm.Bound(pm.Normal, lower=0, upper=1e8)

    U=pm.Beta('U',alpha=1,beta=1)
    B=pm.Beta('B',alpha=1,beta=1)
    mono=np.array(cideal.astype(float))+FluoConc
    A_total=pm.Deterministic('A_total', U + (B-U)*((4*mono + K_D - pm.math.sqrt(pm.math.sqr(4*mono + K_D) - 16*mono**2))/(4*mono)))
    L=pm.Uniform('L',-1,1, shape=cap)
    I=pm.Deterministic('I',1-A_total)
    R=pm.Beta('R',alpha=2,beta=1, shape=cap)
    A_1=pm.Deterministic('A_1',A_total*R)
    A_2=pm.Deterministic('A_2',A_total*(1-R))
    rate1=pm.Lognormal('rate1',0,1, shape=cap)
    rate2=pm.Lognormal('rate2',0,1, shape=cap)

    linearphase=L*(time)
    E_1=A_1*pm.math.exp(-1.0*rate1*time)
    E_2=A_2*pm.math.exp(-1.0*rate2*time)

    epsilon = pm.Lognormal('epsilon', 0, 1)
    laseron=np.tile(np.arange(0, len(onedtime)), (cap,1)).T

    pr = switch(IR >= laseron, 1.0, linearphase+I+E_1+E_2)


    P = pm.Normal('P', mu=pr[start:end], sd=epsilon, observed=fluorescence.astype(float).iloc[start:end])
  
with model:
    trace = pm.sampling.sample(Samples, tune=Tuning,
                      init='ADVI'
                     )
    
with model:
    ppc = pm.sample_posterior_predictive(trace, samples=1000 ,model=model)

Auto-assigning NUTS sampler...
Initializing NUTS using advi...


Convergence achieved at 68400
Interrupted at 68,399 [34%]: Average Loss = -11,725
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [epsilon, rate2, rate1, R, L, B, U, K_D]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 585 seconds.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
There were 7 divergences after tuning. Increase `target_accept` or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The estimated number of effective samples is smaller than 200 for some parameters.
/home/gergely/anaconda3/lib/python3.7/site-packages/pymc3/sampling.py:1618: UserWarning: samples parameter is smaller than nchains ti

In [129]:
#@title STEP 4: Statistics of global variables
#@markdown If you are finished with Step 1-3, the later steps do not have to follow in strict order. <p>
#@markdown K_D and c_fl units are traditionally expressed in nM in the Nanotemper software, U, B and epsilon are expressed as Fnorm.  
pm.stats.summary(trace,var_names=['K_D','B','U', 'epsilon'])

/home/gergely/anaconda3/lib/python3.7/site-packages/arviz/data/io_pymc3.py:91: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
K_D,1405.710,357.100,740.965,2049.770,25.498,18.057,196.0,196.0,183.0,369.0,1.03
B,0.148,0.002,0.144,0.152,0.000,0.000,538.0,538.0,526.0,1571.0,1.01
U,0.056,0.004,0.048,0.063,0.000,0.000,133.0,133.0,135.0,176.0,1.04
epsilon,0.001,0.000,0.001,0.002,0.000,0.000,7847.0,7846.0,7851.0,5191.0,1.00


In [133]:
%matplotlib
#@title STEP 5: Posterior predictive check { run: "auto" }
#@markdown The shaded area represent the High Density Interval (95%) of the posterior predictions.
#@markdown <p> Choose the capillary to analyse:
plt.rcParams.update({'font.size': 16})
plt.subplots(dpi=300,constrained_layout=True)

cmap = plt.cm.get_cmap('cool')
for idx,number in enumerate(["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16"]):
    Cap_display = number #@param ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16"]
    
    color=cmap(idx/16)


    plt.plot(onedtime[start:end],fluorescence[int(Cap_display)-1].astype(float).iloc[start:end], color='gray')
    plt.plot(onedtime[start:end],np.mean(ppc['P'][:,:,int(Cap_display)-1], axis=0),c=color)
    plt.fill_between(np.array(onedtime[start:end],dtype=float), np.array(pm.stats.hpd(ppc['P'][:,:,int(Cap_display)-1])[:,0],dtype=float) ,np.array(pm.stats.hpd(ppc['P'][:,:,int(Cap_display)-1])[:,1],dtype=float) , alpha=0.5,color=color)

plt.xlabel('Time (s)')
plt.ylabel(r'$F_{norm}$')
plt.savefig('PPC.png')

Using matplotlib backend: Qt5Agg


/home/gergely/anaconda3/lib/python3.7/site-packages/arviz/stats/stats.py:338: UserWarning: hpd will be deprecated Please replace hdi
  ("hpd will be deprecated " "Please replace hdi"),
/home/gergely/anaconda3/lib/python3.7/site-packages/arviz/stats/stats.py:487: FutureWarning: hdi currently interprets 2d data as (draw, shape) but this will change in a future release to (chain, draw) for coherence with other functions
  FutureWarning,
/home/gergely/anaconda3/lib/python3.7/site-packages/arviz/stats/stats.py:338: UserWarning: hpd will be deprecated Please replace hdi
  ("hpd will be deprecated " "Please replace hdi"),
/home/gergely/anaconda3/lib/python3.7/site-packages/arviz/stats/stats.py:487: FutureWarning: hdi currently interprets 2d data as (draw, shape) but this will change in a future release to (chain, draw) for coherence with other functions
  FutureWarning,
/home/gergely/anaconda3/lib/python3.7/site-packages/arviz/stats/stats.py:338: UserWarning: hpd will be deprecated Please rep

In [130]:
%matplotlib
#@title STEP 6: Concentration dependence of the kinetic parameters. { run: "auto" }
#@markdown The error bars represent the High Density Interval (95%) of the posterior distribution.<p>
#@markdown Choose the variable name:
ylab={'A_total': r'$A_{total}\ (F_{norm})$',
      'A_1': r'$A_{1}\ (F_{norm})$',
      'A_2': r'$A_{2}\ (F_{norm})$',
      'rate1': r'$\nu_1\ (s^{-1})$',
      'rate2': r'$\nu_2\ (s^{-1})$',
      'L': r'$\nu_0\ (s^{-1})$',
      'I': r'$I\ (F_{norm})$'
     }
for parameter in ["A_total","I", "rate1", "rate2", "A_1", "A_2", "L"]:
    plt.rcParams.update({'font.size': 16})
    fig=plt.subplots(dpi=300,constrained_layout=True)
    plt.errorbar(mono,np.median(trace[parameter],axis=0),yerr=np.abs(pm.hpd(trace[parameter]).T-np.median(trace[parameter],axis=0)))
    plt.xscale('log')
    plt.xlabel('Survivin concentration (nM)')
    plt.ylabel(ylab[parameter])
    plt.savefig(parameter+".png")

Using matplotlib backend: Qt5Agg


/home/gergely/anaconda3/lib/python3.7/site-packages/arviz/stats/stats.py:338: UserWarning: hpd will be deprecated Please replace hdi
  ("hpd will be deprecated " "Please replace hdi"),
/home/gergely/anaconda3/lib/python3.7/site-packages/arviz/stats/stats.py:487: FutureWarning: hdi currently interprets 2d data as (draw, shape) but this will change in a future release to (chain, draw) for coherence with other functions
  FutureWarning,


In [131]:
%matplotlib
#@title STEP 7: Posterior distribution of model parameters (left), parameter trace (right). { run: "auto" }
#@markdown Ideally the parameter traces fluctuate around a stable value.<p> 
#@markdown Choose the parameter name:
parameter = "K_D" #@param ["K_D","B","U","epsilon","A_total","I", "rate1", "rate2", "A_1", "A_2", "L"]
plt.rcParams.update({'font.size': 16})
#fig,ax=plt.subplots()
with model:
  ax=pm.plot_trace(trace[0:],[parameter],combined=True,compact=True)
ax[0,0].set_xlabel(r'$K_d\ (nM)$')
ax[0,0].set_title('')
ax[0,0].set_ylabel(r'$Frequency$')
ax[0,1].set_xlabel(r'$Steps$')
ax[0,1].set_title('')
ax[0,1].set_ylabel(r'$K_d\ (nM)$')
plt.savefig("Kd_traceplot.png",dpi=300)

Using matplotlib backend: Qt5Agg
